In [1]:
import cv2

In [2]:
import mediapipe as mp
import numpy as np
from scipy.signal import lfilter
import matplotlib.pyplot as plt

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
hands = mp_hands.Hands(max_num_hands = 1, static_image_mode = True,min_detection_confidence = 0.5)

In [5]:
# storing all the coordinates at different timesteps in a list
list_x = []
list_y = []
list_z = []
list_avg_x = []
list_avg_y = []
list_avg_z = []


In [6]:
# color and thickness of the lines
color = (255,0,0)
thickness = 2
# threshold after which we should start drawing
threshold = 4

# moving average buffer
k = 5


In [7]:
# canvas on which we are drawing
canvas = np.zeros([512,512,1],dtype=np.uint8)


In [8]:
cap = cv2.VideoCapture(0)
count = 0
while cap.isOpened():
    success, image = cap.read()
    
        
    if not success:
        print("video capture unsuccessful")
        break
    
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    
    
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.multi_hand_landmarks:
        
        
        
        count = count+1
        hand_landmarks = results.multi_hand_landmarks[0]
        
        # add the coordinates to the list
        list_x.append(int(hand_landmarks.landmark[8].x*256))
        list_y.append(int(hand_landmarks.landmark[8].y*256))
        list_z.append(int(hand_landmarks.landmark[8].z*256))
        
        if(count == k):
            avg_x = sum(list_x[-k:])/k
            avg_y = sum(list_y[-k:])/k
            avg_z = sum(list_z[-k:])/k
            
            list_avg_x.append(int(avg_x))
            list_avg_y.append(int(avg_y))
            list_avg_z.append(int(avg_z))
            count = 0
        
            if(len(list_avg_x) > threshold):
                # get the current coordinates as endpoint
                end_point = (list_avg_x[-1], list_avg_y[-1])
                # get the previous coordinates as the start point
                start_point = (list_avg_x[-2], list_avg_y[-2])
                # draw a line joining these points
                cv2.line(canvas,start_point, end_point,color, thickness)
                
                
                
       
        mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    
  
    
    # show the canvas and the webcam image   
    cv2.imshow("Detected Hands", image)
    cv2.imshow("Result", canvas)
    
    
    
    # exit when the user presses 's's
    if cv2.waitKey(5) & 0xFF == ord('s'):
        break
hands.close()
cv2.destroyAllWindows()
cap.release()






In [9]:
print(list_avg_x)

[155, 147, 147, 148, 150, 151, 151, 151, 150, 149, 145, 138, 132, 125, 121, 118, 116, 115, 117, 124, 133, 141, 147, 153, 157, 160, 165, 165, 161, 150, 143, 136, 126, 119, 117, 119, 118, 122, 126, 127, 129, 132]


In [10]:
print(list_avg_y)

[195, 98, 43, 42, 43, 42, 43, 43, 44, 44, 44, 45, 48, 48, 51, 59, 69, 78, 85, 87, 85, 86, 85, 87, 94, 102, 112, 126, 135, 140, 135, 131, 119, 110, 107, 111, 111, 113, 110, 110, 110, 111]
